# BOF

In [1]:
import os
print(os.getcwd())
import gc
gc.collect()

import warnings
warnings.filterwarnings(action='ignore') # 'default'

import gc
gc.collect()

import pandas as pd
import numpy as np

import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from tqdm import tqdm_notebook

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import math
from sklearn.model_selection import train_test_split

from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


np.random.seed(1234)

/Users/baesangjun/Desktop/DACON/빅콘/Before Model


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from tqdm import tqdm_notebook

In [3]:
data = pd.read_csv('data_v3_1.csv')

In [4]:
def coord_block(data_f, block,
                cols, start_x, start_y, seats, vertical=False, inverse=False):
    data_fb = data_f[data_f['block']==block][['seat','floor','box','block','col','seat_num']].drop_duplicates()
    result = []
    
    for col, x, y, s in zip(cols, start_x, start_y, seats):
        df_tmp = data_fb[data_fb['col']==col]
        keys = np.arange(1, s+1, 1)
        if vertical:
            if inverse:
                values = [((x,y-i)) for i in range(s)]
            else:
                values = [(x,y+i) for i in range(s)]
        else:
            if inverse:
                values = [((x-i,y)) for i in range(s)]
            else:
                values = [(x+i,y) for i in range(s)]      
        
        mapping = dict(zip(keys, values))
        df_tmp['coord'] = df_tmp['seat_num'].map(mapping) # map()과 replace의 차이는? 여기서는 replcae()를 먹이면 tuple로 들어가질 않음
        result.append(df_tmp)
        
    return result

In [5]:
def coord_box(data_f, box, mapping):
    data = data_f[data_f['box']==box]
    
    data['coord'] = data['seat_num'].map(mapping)
    
    return data

In [6]:
def e(column_name):
    result = 0
    for char in column_name:
        result = result * 26 + (ord(char) - ord('A') + 1)
    
    return result

# Floor 3

In [7]:
f3 = data[data['floor']==3][['seat','floor','box','block','col','seat_num']].drop_duplicates()

In [8]:
# Box #
f3boxes = []

mapping = {1:(1,81), 2:(2,81), 3:(3,81),
           6:(1,82), 5:(2,82), 4:(3,82)}
f3bx7 = coord_box(f3, 7, mapping)
f3boxes.append(f3bx7)

mapping = {1:(2,84), 2:(3,84), 3:(4,84),
           6:(2,85), 5:(3,85), 4:(4,85),
           7:(2,86), 8:(3,86), 9:(4,86)}
f3bx8 = coord_box(f3, 8, mapping)
f3boxes.append(f3bx8)

mapping = {1:(3,88), 2:(4,88), 3:(5,88),
           6:(3,89), 5:(4,89), 4:(5,89),
           7:(3,90), 8:(4,90), 9:(5,90),
           12:(3,91), 11:(4,91), 10:(5,91)}
f3bx9 = coord_box(f3, 9, mapping)
f3boxes.append(f3bx9)

mapping = {1:(e('CW'),88), 2:(e('CX'),88), 3:(e('CY'),88),
           6:(e('CW'),89), 5:(e('CX'),89), 4:(e('CY'),89),
           7:(e('CW'),90), 8:(e('CX'),90), 9:(e('CY'),90),
           12:(e('CW'),91), 11:(e('CX'),91), 10:(e('CY'),91)}
f3bx10 = coord_box(f3, 10, mapping)
f3boxes.append(f3bx10)

mapping = {1:(e('CX'),84), 2:(e('CY'),84), 3:(e('CZ'),84),
           6:(e('CX'),85), 5:(e('CY'),85), 4:(e('CZ'),85),
           7:(e('CX'),86), 8:(e('CY'),86), 9:(e('CZ'),86)}
f3bx11 = coord_box(f3, 11, mapping)
f3boxes.append(f3bx11)

mapping = {1:(e('CY'),81), 2:(e('CZ'),81), 3:(e('DA'),81),
           6:(e('CY'),82), 5:(e('CZ'),82), 4:(e('DA'),82)}
f3bx12 = coord_box(f3, 12, mapping)
f3boxes.append(f3bx12)

f3bx = pd.concat(f3boxes, axis=0)


# Block #

# Block A
cols = np.arange(1, 8, 1) # 1~7
start_x = [8,8,8,8,10,13,17]
start_y = np.arange(87, 94, 1) # 87~93
seats = [5,5,6,14,12,9,5]

f3bAs = coord_block(data_f=f3, block='A',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f3bA = pd.concat(f3bAs, axis=0)

# Block B
cols = np.arange(4, 8, 1) # 4~7
start_x = [e('W')] * 4
start_y = np.arange(90, 94, 1) # 90~93
seats = [7,7,8,9]

f3bBs = coord_block(data_f=f3, block='B',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f3bB = pd.concat(f3bBs, axis=0)

# Block C
cols = np.arange(4, 8, 1) # 4~7
start_x = [e('AG')] * 4
start_y = np.arange(90, 94, 1) # 90~93
seats = [10,11,12,13]
f3bCs = coord_block(data_f=f3, block='C',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f3bC = pd.concat(f3bCs, axis=0)

# Block D
cols = np.arange(4, 8, 1) # 4~7
start_x = [e('AU')] * 4
start_y = np.arange(90, 94, 1) # 90~93
seats = [12,12,12,13]
f3bDs = coord_block(data_f=f3, block='D',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f3bD = pd.concat(f3bDs, axis=0)

# Block E
cols = np.arange(4, 8, 1) # 4~7
start_x = [e('BL'),e('BK'),e('BJ'),e('BI')]
start_y = np.arange(90, 94, 1) # 90~93
seats = [10,11,12,13]
f3bEs = coord_block(data_f=f3, block='E',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f3bE = pd.concat(f3bEs, axis=0)

# Block F
cols = np.arange(4, 8, 1) # 4~7
start_x = [e('BY'),e('BY'),e('BX'),e('BW')]
start_y = np.arange(90, 94, 1) # 90~93
seats = [7,7,8,9]
f3bFs = coord_block(data_f=f3, block='F',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f3bF = pd.concat(f3bFs, axis=0)

# Block G
cols = np.arange(1, 8, 1) # 1~7
start_x = [e('CP'),e('CP'),
           e('CO'),
           e('CG'),e('CG'),e('CG'),e('CG')]
start_y = np.arange(87, 94, 1) # 87~93
seats = [5,5,6,14,12,9,5]
f3bGs = coord_block(data_f=f3, block='G',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f3bG = pd.concat(f3bGs, axis=0)

# Block M
cols = [1,2]
start_x = [e('AG')] * 2
start_y = [100,101]
seats = [15,15]
f3bMs = coord_block(data_f=f3, block='M',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f3bM = pd.concat(f3bMs, axis=0)

# Block N
cols = [1,2]
start_x = [e('BG')] * 2
start_y = [100,101]
seats = [15,15]
f3bNs = coord_block(data_f=f3, block='N',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f3bN = pd.concat(f3bNs, axis=0)

# Output
f3s = [f3bx,f3bA,f3bB,f3bC,f3bD,f3bE,f3bF,f3bG,f3bM,f3bN]
f3_coord = pd.concat(f3s, axis=0)

# Floor 2

In [9]:
f2 = data[data['floor']==2][['seat','floor','box','block','col','seat_num']].drop_duplicates()

In [10]:
# Box #
f2boxes = []

mapping = {1:(3,63), 2:(4,63),
           4:(3,64), 3:(4,64),
           5:(3,65), 6:(4,65),}
f2bx1 = coord_box(f2, 1, mapping)
f2boxes.append(f2bx1)

mapping = {1:(5,65), 2:(6,65),
           4:(5,66), 3:(6,66),
           5:(5,67), 6:(6,67),
           8:(5,68), 7:(6,68)}
f2bx2 = coord_box(f2, 2, mapping)
f2boxes.append(f2bx2)

mapping = {1:(7,68), 2:(8,68),
           4:(7,69), 3:(8,69),
           5:(7,70), 6:(8,70),
           8:(7,71), 7:(8,71),
           9:(7,72), 10:(8,72)}
f2bx3 = coord_box(f2, 3, mapping)
f2boxes.append(f2bx3)

mapping = {1:(e('CT'),68), 2:(e('CU'),68),
           4:(e('CT'),69), 3:(e('CU'),69),
           5:(e('CT'),70), 6:(e('CU'),70),
           8:(e('CT'),71), 7:(e('CU'),71),
           9:(e('CT'),72), 10:(e('CU'),72)}
f2bx4 = coord_box(f2, 4, mapping)
f2boxes.append(f2bx4)

mapping = {1:(e('CV'),65), 2:(e('CW'),65),
           4:(e('CV'),66), 3:(e('CW'),66),
           5:(e('CV'),67), 6:(e('CW'),67),
           8:(e('CV'),68), 7:(e('CW'),68)}
f2bx5 = coord_box(f2, 5, mapping)
f2boxes.append(f2bx5)

mapping = {1:(e('CX'),63), 2:(e('CY'),63),
           4:(e('CX'),64), 3:(e('CY'),64),
           5:(e('CX'),65), 6:(e('CY'),65)}
f2bx6 = coord_box(f2, 6, mapping)
f2boxes.append(f2bx6)

f2bx = pd.concat(f2boxes, axis=0)


# Block #

# Block A
cols = np.arange(1, 9, 1) # 1~8
start_x = [e('K')] * 8
start_y = np.arange(68, 76, 1) # 68~75
seats = [17,
         18,18,
         19,
         20,
         19,
         16,
         8]
f2bAs = coord_block(data_f=f2, block='A',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f2bA = pd.concat(f2bAs, axis=0)

# Block B
cols = np.arange(1, 9, 1) # 1~8
start_x = [e('AF')] * 8
start_y = np.arange(68, 76, 1) # 68~75
seats = [9,10,11,11,12,13,14,4]
f2bBs = coord_block(data_f=f2, block='B',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f2bB = pd.concat(f2bBs, axis=0)

# Block C
cols = np.arange(1, 8, 1) # 1~7
start_x = [e('AU')] * 7
start_y = np.arange(68, 75, 1) # 68~74
seats = [11,11,11,12,12,12,13]
f2bCs = coord_block(data_f=f2, block='C',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f2bC = pd.concat(f2bCs, axis=0)

# Block D
cols = np.arange(1, 9, 1) # 1~8
start_x = [e('BN'),
           e('BM'),
           e('BL'),e('BL'),
           e('BK'),
           e('BJ'),
           e('BI'),
           e('BS')]
start_y = np.arange(68, 76, 1) # 68~75
seats = [9,10,11,11,12,13,14,4]
f2bDs = coord_block(data_f=f2, block='D',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f2bD = pd.concat(f2bDs, axis=0)

# Block E
cols = np.arange(1, 9, 1) # 1~8
start_x = [e('CA'),
           e('BZ'),e('BZ'),
           e('BY'),
           e('BX'),
           e('BY'),
           e('CB'),
           e('CJ')]
start_y = np.arange(68, 76, 1) # 68~75
seats = [17,18,18,19,20,19,16,8]
f2bEs = coord_block(data_f=f2, block='E',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f2bE = pd.concat(f2bEs, axis=0)

# Output
f2s = [f2bx,f2bA,f2bB,f2bC,f2bD,f2bE]
f2_coord = pd.concat(f2s, axis=0)

# Floor 1

In [11]:
f1 = data[data['floor']==1][['seat','floor','box','block','col','seat_num']].drop_duplicates()

In [12]:
f1s = []

# Block A
cols = np.arange(1, 23, 1) # 1~22
cols = cols[cols!=14] # 1~22, 14열은 없음
start_x = [e('R'),e('R'),e('R'),
           e('Q'),e('Q'),e('Q'),e('Q'),
           e('P'),e('P'),e('P'),e('P'),
           e('O'),e('O'),e('O'),
           e('P'),e('P'),
           e('Q'),
           e('R'),
           e('S'),
           e('T'),e('T'),]
start_y = np.arange(29, 51, 1) # 29~50
start_y = start_y[start_y!=42] # 42열은 없음
seats = [9,9,9,
         10,10,10,10,
         11,11,11,11,
         12,12,12,
         11,11,
         10,
         9,
         8,
         7,7]
f1bAs = coord_block(data_f=f1, block='A',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f1bA = pd.concat(f1bAs, axis=0)
f1s.append(f1bA)

# Block B
cols = np.arange(1, 23, 1) # 1~22
start_x = [e('AG'),
           e('AF'),e('AF'),e('AF'),e('AF'),
           e('AE'),e('AE'),e('AE'),e('AE'),e('AE'),
           e('AD'),e('AD'),e('AD'),e('AD'),
           e('AC'),e('AC'),e('AC'),e('AC'),
           e('AB'),e('AB'),e('AB'),e('AB')]
start_y = np.arange(29, 51, 1) # 29~50
seats = [9,
         10,10,10,10,
         11,11,11,11,11,
         12,12,12,12,
         13,13,13,13,
         14,14,14,14]
f1bBs = coord_block(data_f=f1, block='B',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f1bB = pd.concat(f1bBs, axis=0)
f1s.append(f1bB)

# Block C (22열 제외)
cols = np.arange(1, 22, 1) # 1~21
start_x = [e('AU'),e('AU'),e('AU'),e('AU'),e('AU'),
           e('AT'),e('AT'),e('AT'),e('AT'),e('AT'),
           e('AS'),e('AS'),e('AS'),e('AS'),e('AS'),
           e('AR'),e('AR'),e('AR'),e('AR'),e('AR'),
           e('AQ')]
start_y = np.arange(29, 50, 1) # 29~49
seats=[12,12,12,12,12,
       13,13,13,13,13,
       14,14,14,14,14,
       15,15,15,15,15,
       16]
f1bCs = coord_block(data_f=f1, block='C',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f1bC = pd.concat(f1bCs, axis=0)
f1s.append(f1bC)

# Block C (22열 휠체어석)
f1bC_w = f1[(f1['block']=='C') & (f1['col']==22)]
mapping = {1:(e('AQ'),50), 2:(e('AS'),50), 3:(e('AU'),50), 4:(e('AW'),50),
           5:(e('AY'),50), 6:(e('BA'),50), 7:(e('BC'),50), 8:(e('BE'),50)}
f1bC_w['coord'] = f1bC_w['seat_num'].map(mapping)
f1s.append(f1bC_w)

# Block D
cols = np.arange(1, 23, 1) # 1~22
start_x = [e('BH')] * 22
start_y = np.arange(29, 51, 1) # 29~50
seats=[9,
       10,10,10,10,
       11,11,11,11,11,
       12,12,12,12,
       13,13,13,13,
       14,14,14,14]
f1bDs = coord_block(data_f=f1, block='D',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f1bD = pd.concat(f1bDs, axis=0)
f1s.append(f1bD)

# Block E
cols = np.arange(1, 23, 1) # 1~22
cols = cols[cols!=14] # 1~22, 14열은 없음
start_x = [e('BW')] * 21
start_y = np.arange(29, 51, 1) # 29~50
start_y = start_y[start_y!=42] # 42열은 없음
seats = [9,9,9,
         10,10,10,10,
         11,11,11,11,
         12,12,12,
         11,11,
         10,
         9,
         8,
         7,7]
f1bEs = coord_block(data_f=f1, block='E',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats)
f1bE = pd.concat(f1bEs, axis=0)
f1s.append(f1bE)

# Output
f1_coord = pd.concat(f1s, axis=0)

# Floor 0 (합창석)

In [13]:
f0 = data[data['floor']==0][['seat','floor','box','block','col','seat_num']].drop_duplicates()

In [14]:
f0s = []

# Block H
cols = [1,2,3,4]
start_x = [e('AB'),e('AA'),e('Z'),e('Y')]
start_y = [5,3,1,13]
seats = [16,20,24,12]
f0bHs = coord_block(data_f=f0, block='H',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats, vertical=True)
f0bH = pd.concat(f0bHs, axis=0)
f0s.append(f0bH)

# Block G
cols = [1,2,3,4]
start_x = [e('BL'),e('BM'),e('BN'),e('BO')]
start_y = [4,3,2,1]
seats = [28,31,34,37]
f0bGs = coord_block(data_f=f0, block='G',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats, inverse=True)
f0bG = pd.concat(f0bGs, axis=0)
f0s.append(f0bG)

# Block F
cols = [1,2,3,4]
start_x = [e('BT'),e('BU'),e('BV'),e('BW')]
start_y = [20,22,24,24]
seats = [16,20,24,12]
f0bFs = coord_block(data_f=f0, block='F',
                    cols=cols, start_x=start_x, start_y=start_y, seats=seats, vertical=True, inverse=True)
f0bF = pd.concat(f0bFs, axis=0)
f0s.append(f0bF)

# Output
f0_coord = pd.concat(f0s, axis=0)

In [15]:
# Final Output
coord_all = pd.concat([f0_coord,f1_coord,f2_coord,f3_coord], axis=0)

# Additional Adjustment

In [16]:
xy = pd.DataFrame(coord_all['coord'].tolist(), index=coord_all.index, columns=['x','y'])
coord_all = pd.concat([coord_all,xy], axis=1)

In [17]:
coord_all[coord_all['coord'].isna()] # 좌석 배치표에 존재하지 않는 구역

,seat,floor,box,block,col,seat_num,coord,x,y
39638,2층 A블록8열 12,2,NaN,A,8.0,12,NaN,NaN,NaN
97589,2층 A블록8열 13,2,NaN,A,8.0,13,NaN,NaN,NaN
4038,2층 E블록8열 13,2,NaN,E,8.0,13,NaN,NaN,NaN
147839,2층 E블록8열 12,2,NaN,E,8.0,12,NaN,NaN,NaN


In [18]:
coord_all.loc[(coord_all['floor']==2) &
              (coord_all['block']=='A') &
              (coord_all['col']==8) &
              (coord_all['seat_num']==12),'x'] = 22
coord_all.loc[(coord_all['floor']==2) &
              (coord_all['block']=='A') &
              (coord_all['col']==8) &
              (coord_all['seat_num']==12),'y'] = 20
coord_all.loc[(coord_all['floor']==2) &
              (coord_all['block']=='A') &
              (coord_all['col']==8) &
              (coord_all['seat_num']==13),'x'] = 23
coord_all.loc[(coord_all['floor']==2) &
              (coord_all['block']=='A') &
              (coord_all['col']==8) &
              (coord_all['seat_num']==13),'y'] = 20
coord_all.loc[(coord_all['floor']==2) &
              (coord_all['block']=='E') &
              (coord_all['col']==8) &
              (coord_all['seat_num']==12),'x'] = 94
coord_all.loc[(coord_all['floor']==2) &
              (coord_all['block']=='E') &
              (coord_all['col']==8) &
              (coord_all['seat_num']==12),'y'] = 20
coord_all.loc[(coord_all['floor']==2) &
              (coord_all['block']=='E') &
              (coord_all['col']==8) &
              (coord_all['seat_num']==13),'x'] = 95
coord_all.loc[(coord_all['floor']==2) &
              (coord_all['block']=='E') &
              (coord_all['col']==8) &
              (coord_all['seat_num']==13),'y'] = 20

In [19]:
coord_all = coord_all.drop(columns='coord')

# Final Output

In [20]:
data1 = data.merge(coord_all, how='left')

In [21]:
print(data.shape, data1.shape)

(1051394, 39) (1051394, 41)


In [22]:
data1.to_csv('data_v3_2_xy.csv', index=False)